In [1]:
#Torch was already installed - you might need to figure out torch + cuda setup for your GPU

!pip install git+https://github.com/openai/CLIP.git
!python -m pip install pytube

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-49o0a8du
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-49o0a8du
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
     |████████████████████████████████| 53 kB 98 kB/s  eta 0:00:011
     |████████████████████████████████| 752 kB 4.9 MB/s eta 0:00:01
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369400 sha256=569a5ca4854ee3c4b5f1900600d7926e5b0e251c2e3eefad843f937cc35c5984
  Stored in directory: /tmp/pip-ephem-wheel-cache-jv9t9h9b/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [3]:
from pytube import Channel
import re
import os

In [4]:
c = Channel("https://www.youtube.com/c/DaveThielking/videos")

In [98]:
for vid in c.videos:
    if vid.title.startswith("90"):
        vid.streams.filter(progressive=True, file_extension='mp4').first().download("videos", re.sub('[^A-Za-z0-9]+', '_', vid.title))

In [ ]:
for file in os.listdir("videos"):
    cmd = f"ffmpeg -i videos/{file} -r 0.5 images/{file}_%04d.png"
    print(file)
    print(cmd)
    if file.startswith("."):
        continue
    ! $cmd

In [41]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:13<00:00, 26.7MiB/s]


In [100]:
image_names = [f'images/{file}' for file in sorted(os.listdir("images")) if not file.startswith(".")]

In [101]:
grouped_image_names = {}

for image_name in image_names:
    g = image_name.split("_")[-2]
    if g in grouped_image_names:
        grouped_image_names[g].append(image_name)
    else:
        grouped_image_names[g] = [image_name]

In [135]:
all_matches = []

source_image = Image.open("source_img.jpeg")
source_image_vit = preprocess(source_image).unsqueeze(0)

for (k,v) in grouped_image_names.items():
    vitd = []
    for image in v:
        vitd.append(preprocess(Image.open(image)).unsqueeze(0))
        
    #text = clip.tokenize(["an old cartoon elf with a beard"]).to(device)
    t = torch.cat(vitd, dim=0).to(device)
    source_image_vit = source_image_vit.to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(t)
        #text_features = model.encode_text(text)
        search_image_features = model.encode_image(source_image_vit)

        # change the next line to "text" or "source_image" depending on what you want
        
        image_features /= image_features.norm(dim=-1, keepdim=True)
        search_image_features /= search_image_features.norm(dim=-1, keepdim=True)
        similarity = image_features.cpu().numpy() @ search_image_features.cpu().numpy().T
        
        # these arent actually probs anymore, they're angles btw the vectors
        probs = similarity
        #logits_per_image, logits_per_text = model(t, text)
        #probs = logits_per_image.cpu().numpy()

        
    # it says probability, but its actually an angle now!
    log_probs = [x[0] for x in probs]

    top_20 = sorted(range(len(log_probs)), key=lambda i: log_probs[i])[-20:]
    
    print(f"Processed Volume #{k}, Highest Log Prob: {log_probs[top_20[-1]]}, Best frame:, {v[top_20[-1]]}")
    if (log_probs[top_20[-1]] > 0.7):
        print("-------")
        print("MATCH > 0.7:")
        for i in range(1,19):
            if (log_probs[top_20[-i]]) > 0.7:
                print(f"Log Prob: {log_probs[top_20[-i]]} Frame: {v[top_20[-i]]}")
                all_matches.append((log_probs[top_20[-i]], v[top_20[-i]]))
    print("------")

Processed Volume #100, Highest Log Prob: 0.6337890625, Best frame:, images/90_s_Commercials_Ontario_Vol_100_0188.png
------
Processed Volume #101, Highest Log Prob: 0.681640625, Best frame:, images/90_s_Commercials_Ontario_Vol_101_0062.png
------
Processed Volume #102, Highest Log Prob: 0.72607421875, Best frame:, images/90_s_Commercials_Ontario_Vol_102_0153.png
-------
MATCH > 0.7:
Log Prob: 0.72607421875 Frame: images/90_s_Commercials_Ontario_Vol_102_0153.png
------
Processed Volume #103, Highest Log Prob: 0.67236328125, Best frame:, images/90_s_Commercials_Ontario_Vol_103_0256.png
------
Processed Volume #104, Highest Log Prob: 0.6513671875, Best frame:, images/90_s_Commercials_Ontario_Vol_104_0083.png
------
Processed Volume #105, Highest Log Prob: 0.66552734375, Best frame:, images/90_s_Commercials_Ontario_Vol_105_0205.png
------
Processed Volume #106, Highest Log Prob: 0.7412109375, Best frame:, images/90_s_Commercials_Ontario_Vol_106_0098.png
-------
MATCH > 0.7:
Log Prob: 0.741

In [136]:
[x[1] for x in sorted(all_matches, key=lambda i: -i[0])[:36]]


['images/90_s_Commercials_Ontario_Vol_57_0214.png',
 'images/90_s_Commercials_Ontario_Vol_137_0234.png',
 'images/90_s_Commercials_Ontario_Vol_4_0026.png',
 'images/90_s_Commercials_Ontario_Vol_88_0107.png',
 'images/90_s_Commercials_Ontario_Vol_57_0218.png',
 'images/90_s_Commercials_Ontario_Vol_55_0181.png',
 'images/90_s_Commercials_Ontario_Vol_65_0174.png',
 'images/90_s_Commercials_Ontario_Vol_110_The_Sears_Edition_0119.png',
 'images/90_s_Commercials_Ontario_Vol_139_0124.png',
 'images/90_s_Commercials_Ontario_Vol_141_0411.png',
 'images/90_s_Commercials_Ontario_Vol_93_0189.png',
 'images/90_s_Commercials_Ontario_Vol_65_0175.png',
 'images/90_s_Commercials_Ontario_Vol_142_0204.png',
 'images/90_s_Commercials_Ontario_Vol_93_0080.png',
 'images/90_s_Commercials_Ontario_Vol_181_0137.png',
 'images/90_s_Commercials_Ontario_Vol_197_0284.png',
 'images/90_s_Commercials_Ontario_Vol_121_0119.png',
 'images/90_s_Commercials_Ontario_Vol_93_0200.png',
 'images/90_s_Commercials_Ontario_Vol_9